In [1]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize

In [2]:
!pip3 install python-dotenv

### 1. Importo mi dataframe

In [3]:
df= pd.read_csv("../data/Food and Calories.csv")

In [4]:
#url = "https://www.fruityvice.com/api/fruit/all"

In [5]:
#response = requests.get(url)

In [6]:
#response

In [7]:
#res_js= response.json()

In [8]:
#res_js

In [9]:
#len(res_js)

In [10]:
#res_js[0]

In [11]:
df.head()

,Food,Serving,Calories
0,Artichoke,1 artichoke (128 g),60 cal
1,Arugula,1 leaf (2 g),1 cal
2,Asparagus,1 spear (12 g),2 cal
3,Aubergine,1 aubergine (458 g),115 cal
4,Beetroot,1 beet (82 g),35 cal


### Limpio mi data frame eliminando nulos y duplicados

In [12]:
df.shape

(562, 3)

In [13]:
df.dropna(how="all", inplace=True)

In [14]:
df.shape #no hay ninguna fila entera de NaN

(562, 3)

In [15]:
duplicates= df.drop_duplicates() #elimino los duplicados

In [16]:
duplicates.shape

(549, 3)

In [17]:
food = duplicates["Food"].unique()

### Obtengo una lista con los valores únicos de la columna Food que serán los valores de mi dataframe que utilizaré con la información de la API con la que trabaje a continuación

In [18]:
lista_food= list(food)

In [19]:
type(lista_food)

list

In [63]:
lista_food

['Artichoke',
 'Arugula',
 'Asparagus',
 'Aubergine',
 'Beetroot',
 'Bell Pepper',
 'Black Olives',
 'Broccoli',
 'Brussels Sprouts',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Celery',
 'Chard',
 'Cherry Tomato',
 'Chicory',
 'Chinese Cabbage',
 'Chives',
 'Collard Greens',
 'Corn',
 'Courgette',
 'Creamed Spinach',
 'Cucumber',
 'Eggplant',
 'Endive',
 'Fennel',
 'Garlic',
 'Gherkin',
 'Gourd',
 'Green Beans',
 'Green Olives',
 'Green Onion',
 'Horseradish',
 'Kale',
 'Kohlrabi',
 'Kumara',
 'Leek',
 'Lettuce',
 'Mushrooms',
 'Mustard Greens',
 'Nori',
 'Okra',
 'Olives',
 'Onion',
 'Parsnips',
 'Peas',
 'Pepper',
 'Potato',
 'Pumpkin',
 'Radishes',
 'Red Cabbage',
 'Rutabaga',
 'Shallots',
 'Spinach',
 'Squash',
 'Sweet Potato',
 'Tomato',
 'Turnip Greens',
 'Turnips',
 'Wasabi',
 'Winter Squash',
 'Zucchini',
 'Acai',
 'Apple',
 'Applesauce',
 'Apricot',
 'Avocado',
 'Banana',
 'Blackberries',
 'Blood Oranges',
 'Blueberries',
 'Cantaloupe',
 'Cherries',
 'Clementine',
 

In [64]:
type(lista_food)

list

In [ ]:
#tengo una url de API pero la llamo concretamente para un alimento, hago una PRUEBA para que ver qué me devuelve introduciendo un alimento que sé (porque está en español) que no va a estar en la lista de APIs.

In [52]:
#url_nut = 'https://api.edamam.com/api/nutrition-data?app_id=5bb5f3ad&app_key=a3a514dbb62d9ea04d57160922fce573&nutrition-type=logging&ingr=manzana'

In [53]:
#apple = requests.get(url_nut)

In [56]:
#manzana = apple.json()

In [57]:
#df = pd.DataFrame.from_dict(manzana, orient="index")

In [58]:
#df. Me devuelve un dataframe con los valores vacío pero no me da error.

,0
uri,http://www.edamam.com/ontologies/edamam.owl#re...
calories,0
totalWeight,0.0
dietLabels,[]
healthLabels,"[LOW_SUGAR, VEGAN, VEGETARIAN, PESCATARIAN, PA..."
cautions,[]
totalNutrients,{}
totalDaily,{}
totalNutrientsKCal,"{'ENERC_KCAL': {'label': 'Energy', 'quantity':..."


In [84]:
#url2= 'https://api.edamam.com/api/nutrition-data'

### Me devuelve un json pero cuando meto la url entera con los 3 parámetros, app_id, app_key, ingr ; pero solo me devuelve un 200 cuando meto un ingrediente concreto, en este caso "apple"

In [90]:
url2 = 'https://api.edamam.com/api/nutrition-data?app_id=5bb5f3ad&app_key=a3a514dbb62d9ea04d57160922fce573&nutrition-type=logging&ingr=apple'

In [91]:
parameters = {"app_id": '5bb5f3ad', "app_key":'a3a514dbb62d9ea04d57160922fce573', "ingr":"apple"}

In [92]:
response2 = requests.get(url2, params=parameters)

In [93]:
response2

<Response [200]>

In [94]:
api=response2.json()

In [95]:
api #me da los valores de apple

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_fb479049500e45e5bcec0174de10e0fe',
 'calories': 125,
 'totalWeight': 242.0,
 'dietLabels': ['LOW_FAT', 'LOW_SODIUM'],
 'healthLabels': ['FAT_FREE',
  'LOW_FAT_ABS',
  'LOW_POTASSIUM',
  'KIDNEY_FRIENDLY',
  'VEGAN',
  'VEGETARIAN',
  'PESCATARIAN',
  'PALEO',
  'SPECIFIC_CARBS',
  'MEDITERRANEAN',
  'DASH',
  'DAIRY_FREE',
  'GLUTEN_FREE',
  'WHEAT_FREE',
  'EGG_FREE',
  'MILK_FREE',
  'PEANUT_FREE',
  'TREE_NUT_FREE',
  'SOY_FREE',
  'FISH_FREE',
  'SHELLFISH_FREE',
  'PORK_FREE',
  'RED_MEAT_FREE',
  'CRUSTACEAN_FREE',
  'CELERY_FREE',
  'MUSTARD_FREE',
  'SESAME_FREE',
  'LUPINE_FREE',
  'MOLLUSK_FREE',
  'ALCOHOL_FREE',
  'NO_OIL_ADDED',
  'NO_SUGAR_ADDED',
  'SULPHITE_FREE',
  'KOSHER'],
 'cautions': ['SULFITES'],
 'totalNutrients': {'ENERC_KCAL': {'label': 'Energy',
   'quantity': 125.84,
   'unit': 'kcal'},
  'FAT': {'label': 'Total lipid (fat)',
   'quantity': 0.41140000000000004,
   'unit': 'g'},
  'FASAT': {'label': 

In [28]:
type(api)

dict

### Voy a hacer una prueba con una lista de 3 ingredientes, para que en la función me itere por la lista. De esos tres ingredientes se que uno no va a estar en la api porque está en español pero los otros dos si.

In [59]:
lista2 = ["manzana", "pear", "lemon"]

In [75]:
def call_api(app_id, app_key, ingredientes):
    result_list = []
    for ingre in ingredientes: 
        b = requests.get(f'https://api.edamam.com/api/nutrition-data?app_id={app_id}&app_key={app_key}&nutrition-type=logging&ingr={ingre}').content
        r = json.loads(b.decode())
        result_list.append(r)
    return result_list

In [76]:
prueba = call_api('5bb5f3ad','a3a514dbb62d9ea04d57160922fce573',lista2)

In [77]:
pd.DataFrame(prueba)

,uri,calories,totalWeight,dietLabels,healthLabels,cautions,totalNutrients,totalDaily,totalNutrientsKCal
0,http://www.edamam.com/ontologies/edamam.owl#re...,0,0.0,[],"[LOW_SUGAR, VEGAN, VEGETARIAN, PESCATARIAN, PA...",[],{},{},"{'ENERC_KCAL': {'label': 'Energy', 'quantity':..."
1,http://www.edamam.com/ontologies/edamam.owl#re...,94,166.0,"[LOW_FAT, LOW_SODIUM]","[FAT_FREE, LOW_FAT_ABS, LOW_POTASSIUM, KIDNEY_...",[SULFITES],"{'ENERC_KCAL': {'label': 'Energy', 'quantity':...","{'ENERC_KCAL': {'label': 'Energy', 'quantity':...","{'ENERC_KCAL': {'label': 'Energy', 'quantity':..."
2,http://www.edamam.com/ontologies/edamam.owl#re...,16,58.0,"[LOW_FAT, LOW_SODIUM]","[FAT_FREE, LOW_FAT_ABS, SUGAR_CONSCIOUS, LOW_P...",[SULFITES],"{'ENERC_KCAL': {'label': 'Energy', 'quantity':...","{'ENERC_KCAL': {'label': 'Energy', 'quantity':...","{'ENERC_KCAL': {'label': 'Energy', 'quantity':..."


### me devuelve un df con los valores de manzana vacíos y los valores de los otros dos ingredientes que si se encontraban en la API completados.

### pero cuando le incluyo mi lista_food de ingredientes de mi df, y modifico la url poniendo en el parámetro de ingr, en lugar de apple, {ingre}, se queda colgado.

In [ ]:
#def call_api(app_id, app_key, lista_food):
    result_list = []
    for ingre in lista_food: 
        b = requests.get(f'https://api.edamam.com/api/nutrition-data?app_id={app_id}&app_key={app_key}&nutrition-type=logging&ingr={ingre}').content
        r = json.loads(b.decode())
        result_list.append(r)
    return result_list